# Friedmannove jednadžbe

Ovo je prilagodba radnog listića dostupnog [ovdje](http://sagemanifolds.obspm.fr/examples.html). 

Da bi sve funkcioniralo, osim jupyter notebooka potrebno je instalirati i Sage kernel. To se na Arch linuxu može tim redom. Drugdje je možda pogodnije naprosto instalirati Sage koji mislim dolazi sa svojim jupyter notebookom.

In [1]:
version()

'SageMath version 8.1, Release Date: 2017-12-07'

Postavljanje prikaza simbola korištenjem LaTeX formatiranja:

In [2]:
%display latex

Deklariramo prostorvrijeme kao 4-dimenzionalnu diferencijabilnu mnogostrukost $M$:

In [3]:
M = Manifold(4, 'M')
print(M)

4-dimensional differentiable manifold M


Uvodimo standardne (FL)RW koordinate, koristeći metodu `chart()`, s argumentom koji je python string s razmaknuto navedenim koordinatama u sintaksi
`simbol:raspon vrijednosti:LaTeX simbol`
gdje je defaultni raspon vrijednosti $(-\infty, \infty)$:

In [4]:
fr.<t,r,th,ph> = M.chart(r't r:[0,+oo) th:[0,pi]:\theta ph:[0,2*pi):\phi')
fr

Chart (M, (t, r, th, ph))

Definiramo skalarne varijable: Newtonovu konstantu $G$, konstantu prostorne zakrivljenosti $k\in\{1,0,-1\}$, radijus zakrivljenosti $R_{0}$, faktor skale $a(t)$, gustoća energije kozmičkog fluida $\epsilon(t)$  i njegov tlak $p(t)$. Za RW metriku koristimo izraz iz Ryden (2016)

$$ ds^2 = c^2 dt^2 - a^{2}(t)\left(\frac{dr^2}{1-\frac{k r^2}{R_{0}^2}} + r^2 d\Omega\right) $$
  
uz napomenu da koristimo suprotni predznak metrike i da većina literature apsorbira radijus zakrivljenosti $R_0$ ili u konstantu zakrivljenosti $K \equiv k/R_{0}^2$ ili u faktor skale uz redefiniciju koordinate $r\to r R_0$ tako da postane bezdimenzionalna.

In [5]:
var('G, Lambda, k, R0, c', domain='real')
a = M.scalar_field(function('a')(t), name='a')
epsilon = M.scalar_field(function('epsilon')(t), name='epsilon')
p = M.scalar_field(function('p')(t), name='p')

RW metriku definiramo u "-" konvenciji (tj. "čestičarska" ili "West coast", dakle obrnuto od Ryden). Također, budući da je gore nulta koordinata izabrana kao $t$, a ne, kako je uobičajeno $ct$, moramo uzeti $g_{00}=c^2$, a ne, kako je uobičajeno 1.

In [6]:
g = M.lorentzian_metric('g', signature='negative')
g[0,0] = c*c
g[1,1] = -a*a/(1 - k*r^2/R0^2)
g[2,2] = -a*a*r^2
g[3,3] = -a*a*(r*sin(th))^2
g.display()

g = c^2 dt*dt + a(t)^2/(k*r^2/R0^2 - 1) dr*dr - r^2*a(t)^2 dth*dth - r^2*a(t)^2*sin(th)^2 dph*dph

<p>Matrični zapis:</p>

In [7]:
g[:]

[                    c^2                       0                       0                       0]
[                      0 a(t)^2/(k*r^2/R0^2 - 1)                       0                       0]
[                      0                       0             -r^2*a(t)^2                       0]
[                      0                       0                       0   -r^2*a(t)^2*sin(th)^2]

<p>Christoffelovi simboli su kvadratni u metričkom tenzoru pa ne ovise o konvencijama za predznake:</p>

In [8]:
nabla = g.connection()
g.christoffel_symbols_display()

Gam^t_r,r = -R0^2*a(t)*d(a)/dt/(c^2*k*r^2 - R0^2*c^2) 
Gam^t_th,th = r^2*a(t)*d(a)/dt/c^2 
Gam^t_ph,ph = r^2*a(t)*sin(th)^2*d(a)/dt/c^2 
Gam^r_t,r = d(a)/dt/a(t) 
Gam^r_r,r = -k*r/(k*r^2 - R0^2) 
Gam^r_th,th = (k*r^3 - R0^2*r)/R0^2 
Gam^r_ph,ph = (k*r^3 - R0^2*r)*sin(th)^2/R0^2 
Gam^th_t,th = d(a)/dt/a(t) 
Gam^th_r,th = 1/r 
Gam^th_ph,ph = -cos(th)*sin(th) 
Gam^ph_t,ph = d(a)/dt/a(t) 
Gam^ph_r,ph = 1/r 
Gam^ph_th,ph = cos(th)/sin(th)

Riccijev tenzor:

In [9]:
Ricci = nabla.ricci()
Ricci.display_comp()

Ric(g)_t,t = -3*d^2(a)/dt^2/a(t) 
Ric(g)_r,r = -(2*R0^2*(d(a)/dt)^2 + R0^2*a(t)*d^2(a)/dt^2 + 2*c^2*k)/(c^2*k*r^2 - R0^2*c^2) 
Ric(g)_th,th = (2*R0^2*r^2*(d(a)/dt)^2 + R0^2*r^2*a(t)*d^2(a)/dt^2 + 2*c^2*k*r^2)/(R0^2*c^2) 
Ric(g)_ph,ph = (2*R0^2*r^2*(d(a)/dt)^2 + R0^2*r^2*a(t)*d^2(a)/dt^2 + 2*c^2*k*r^2)*sin(th)^2/(R0^2*c^2)

Ovi izrazi su, nakon zamjene $k \to k R_0$, u slaganju s [wikipedijom](https://en.wikipedia.org/wiki/Friedmann%E2%80%93Lema%C3%AEtre%E2%80%93Robertson%E2%80%93Walker_metric#Spherical_coordinates):

$$ R_{tt} = - 3 \frac{\ddot{a}}{a}, \\
R_{rr}=\frac{c^{-2}(a(t)\ddot{a}(t) + 2\dot{a}^2(t)) + 2k}{1 - kr^2} \\
R_{\theta\theta} = r^2(c^{-2}(a(t)\ddot{a}(t) + 2\dot{a}^2(t)) + 2k) \\
R_{\phi\phi} =r^2(c^{-2}(a(t)\ddot{a}(t) + 2\dot{a}^2(t)) + 2k)\sin^2(\theta)$$

Iz dokumentacije SageManifolds paketa može se zaključiti da isti koristi "+" konvenciju za prezdnak Riemannovog tenzora, kao i "+" konvenciju za vezu Riccijevog i Riemannovog tenzora (gdje za razliku od gornje konvencije za metriku korisnik to ne može mijenjati), što znači da je nužno pisati Einsteinovu jednadžbu gravitacije u "+" konvenciji, tj.
$$ R_{\mu\nu} - \frac{1}{2}g_{\mu\nu}R = + \frac{8\pi G}{c^4} T_{\mu\nu}$$

<p>Riccijev skalar ($R^\mu_{\ \, \mu}$):</p>

In [10]:
Ricci_scalar = g.ricci_scalar()
Ricci_scalar.display()

r(g): M --> R
   (t, r, th, ph) |--> -6*(R0^2*(d(a)/dt)^2 + R0^2*a(t)*d^2(a)/dt^2 + c^2*k)/(R0^2*c^2*a(t)^2)

(Za razliku od Riccijevog tenzora koji je invarijantan na odabir konvencije za metrički tenzor, Riccijev skalar bi u drugoj konvenciji za metrički tenzor imao obrnuti predznak, npr. na gornjoj stranici wikipedije.) Uočavamo da je za statičko prostorvrijeme Riccijev skalar

In [11]:
Ricci_scalar.expr().subs({a.expr():1, diff(a.expr(),t):0, diff(a.expr(),t,2):0})

-6*k/R0^2

dakle proporcionalan je Gaussovoj zakrivljenosti plohe (relativni minus je u ovoj konvenciji za metriku jer je zakrivljen prostorni dio). To je manifestacija Gaussovog "teorema egregium" po kojem je ekstrinsično svojstvo plohe (radijus zakrivljenosti) potpuno određeno intrinsičnom metrikom na plohi.

<p>4-brzina kozmičkog fluida:</p>

In [12]:
u = M.vector_field('u')
u[0] = 1/c
u.display()

u = 1/c d/dt

<p>Tenzor energije-impulsa $T$ za savršeni fluid. (Uočite da gustoća energije i tlak imaju istu dimenziju ${\rm N/m}^2$):</p>

In [13]:
u_form = u.down(g) # the 1-form associated to u by metric duality
T = (epsilon+p)*(u_form*u_form) - p*g
T.set_name('T')
print(T)
T.display()

Field of symmetric bilinear forms T on the 4-dimensional differentiable manifold M


T = c^2*epsilon(t) dt*dt - R0^2*a(t)^2*p(t)/(k*r^2 - R0^2) dr*dr + r^2*a(t)^2*p(t) dth*dth + r^2*a(t)^2*p(t)*sin(th)^2 dph*dph

<p>Trag od $T$:</p>

In [14]:
Ttrace = g.inverse()['^ab']*T['_ab']
Ttrace.display()

M --> R
(t, r, th, ph) |--> epsilon(t) - 3*p(t)

Einsteinova jednadžba gravitacije. Kako je gore navedeno, nužno u "+" konvenciji: $R_{\mu \nu} - {1 \over 2} R g_{\mu \nu} = \frac{8 \pi G}{c^4} T_{\mu \nu}$. Komponenta nula-nula daje **Friedmannovu jednadžbu**:

In [15]:
E1 = Ricci - Ricci_scalar/2*g - (8*pi*G/c^4)*T
(E1[0,0]/3).expr().expand() == 0      # dividing everything by 3

-8/3*pi*G*epsilon(t)/c^2 + diff(a(t), t)^2/a(t)^2 + c^2*k/(R0^2*a(t)^2) == 0

Ekvivalentan, drugačiji oblik Einsteinove jednadžbe, $R_{\mu \nu} = {8 \pi G} \left(T_{\mu \nu} - {1 \over 2}T\,g_{\mu \nu}\right)$, odmah daje **jednadžbu ubrzanja**:

In [16]:
E2 = Ricci - (8*pi*G/c^4)*(T - Ttrace/2*g)
(E2[0,0]/3).expr().expand() == 0

-4/3*pi*G*epsilon(t)/c^2 - 4*pi*G*p(t)/c^2 - diff(a(t), t, t)/a(t) == 0

Ovo se slaže s jednadžbama (4.20) i (4.49) u Ryden (2nd ed.), odnosno (4.13) i (4.44) u Ryden (1st ed.).

Ako želimo izdvojiti doprinos kozmološke konstante $R_{\mu \nu} - {1 \over 2} R g_{\mu \nu} - \Lambda g_{\mu\nu} = \frac{8 \pi G}{c^4} T_{\mu \nu}$:

In [17]:
E1 = Ricci - Ricci_scalar/2*g - Lambda*g - (8*pi*G/c^4)*T
(E1[0,0]/3).expr().expand() == 0      # dividing everything by 3

-1/3*Lambda*c^2 - 8/3*pi*G*epsilon(t)/c^2 + diff(a(t), t)^2/a(t)^2 + c^2*k/(R0^2*a(t)^2) == 0

In [18]:
E2 = Ricci + Lambda*g - (8*pi*G/c^4)*(T - Ttrace/2*g)
(E2[0,0]/3).expr().expand() == 0

1/3*Lambda*c^2 - 4/3*pi*G*epsilon(t)/c^2 - 4*pi*G*p(t)/c^2 - diff(a(t), t, t)/a(t) == 0